In [1]:
import ROOT

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x56c0120


In [2]:
%jsroot on

In [3]:
ROOT.EnableImplicitMT(6)
ROOT.TH1.SetDefaultSumw2()

In [4]:
ROOT.ildStyle.SetOptStat(1)
ROOT.ildStyle.SetPalette(ROOT.kBird)

In [5]:
%%cpp
using namespace ROOT::VecOps;

In [6]:
ROOT.gInterpreter.Declare("#include <podio/DataSource.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/dataframe.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/kinematics.h>")
ROOT.gInterpreter.Declare("#include <edm4hep/utils/vector_utils.h>")
ROOT.gSystem.Load("libedm4hep")
_edm  = ROOT.edm4hep.ReconstructedParticleData()

In [7]:
# df = ROOT.RDataFrame("events", "data/truejet/test_dst/eLpR_15176_0.truejet.edm4hep.root")
# df = ROOT.podio.CreateDataFrame("data/truejet/test_dst/eLpR_15176_0.truejet.edm4hep.root")
df = ROOT.podio.CreateDataFrame("root://eosuser.cern.ch//eos/user/l/lreichen/paris/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I500105.P4f_sw_sl.eL.pL.n000.d_dstm_15065_0.miniDST.edm4hep.root")

In [10]:
df.Describe()

Dataframe from datasource Custom Datasource

Property                Value
--------                -----
Columns in total          106
Columns from defines        0
Event loops run             0
Processing slots            6

Column                                          Type                                            Origin
------                                          ----                                            ------
BCalClusters                                    edm4hep::ClusterCollection                      Dataset
BCalRecoParticle                                edm4hep::ReconstructedParticleCollection        Dataset
BCalRecoParticle_startVertices                  edm4hep::VertexRecoParticleLinkCollection       Dataset
BuildUpVertex                                   edm4hep::VertexCollection                       Dataset
BuildUpVertex_RP                                edm4hep::ReconstructedParticleCollection        Dataset
BuildUpVertex_RP_startVertices                  

In [11]:
%%cpp
ROOT::Math::PxPyPzEVector make_PxPyPzEVec(edm4hep::Vector3f P, float E)
{
    return ROOT::Math::PxPyPzEVector(P.x, P.y, P.z, E);
}

In [12]:
from math import sin
# stuff that could be arguments or read from file parameters
sqrt_s = 250 # GeV
x_angle = 0.014 # rad

p_x = 250 * sin(x_angle/2)

In [13]:
df = df.Alias("ReconstructedParticles", "PandoraPFOs")
# df = df.Alias("ReconstructedParticles", "PFOsWithoutMCOverlay")
df = df.Define("reco_lvecs", "Map(RVec(ReconstructedParticles.momentum()), RVec(ReconstructedParticles.energy()), make_PxPyPzEVec)")
# df = df.Define("PFOsFromOverlay_theta", "return Map(PFOsFromOverlay_lvecs, [] (const auto& el) {return el.Theta();});")
# df = df.Define("PFOsFromOverlay_p", "return Map(PFOsFromOverlay_lvecs, [] (const auto& el) {return el.P();});")
df = df.Define("reco_p", "return Map(reco_lvecs, [] (const auto& el) {return el.P();});")
df = df.Define("leading_idx", "ArgMax(reco_p)")
df = df.Define("leading_pfo", "ReconstructedParticles[leading_idx]")
df = df.Define("leading_lvec", "reco_lvecs[leading_idx]")
df = df.Define("initial_lvec", f"ROOT::Math::PxPyPzEVector({p_x}, 0., 0., {sqrt_s})")
# would go to stage2 after un-boosting everything?
df = df.Define("nu_lvec", "initial_lvec - Sum(reco_lvecs, ROOT::Math::PxPyPzEVector())")
df = df.Define("nu_ptcut_lvec", "return initial_lvec - Sum(Filter(reco_lvecs, [] (const auto& el) {return el.Pt() > 0.5;}), ROOT::Math::PxPyPzEVector());")

df = df.Define("nu_e", "nu_lvec.E()")
df = df.Define("nu_ptcut_e", "nu_ptcut_lvec.E()")
df = df.Define("leading_e", "leading_lvec.E()")
df = df.Define("nu_m", "nu_lvec.M()")
df = df.Define("nu_ptcut_m", "nu_ptcut_lvec.M()")

In [14]:
h_nu_e = df.Histo1D("nu_e")
h_nu_ptcut_e = df.Histo1D("nu_ptcut_e")
h_nu_m = df.Histo1D("nu_m")
h_nu_ptcut_m = df.Histo1D("nu_ptcut_m")
h_leading_e = df.Histo1D("leading_e")
h_2d_leading_e_nu_e = df.Histo2D(("", ";E_{e} [GeV];E_{miss} [GeV]", 150, 0., 150., 200, -50., 150.), "leading_e", "nu_e")

In [15]:
c_nu_e = ROOT.TCanvas()
h_nu_e.Draw()
c_nu_e.Draw()

c_nu_ptcut_e = ROOT.TCanvas()
h_nu_ptcut_e.Draw()
c_nu_ptcut_e.Draw()

c_nu_m = ROOT.TCanvas()
h_nu_m.Draw()
c_nu_m.Draw()

c_nu_ptcut_m = ROOT.TCanvas()
h_nu_ptcut_m.Draw()
c_nu_ptcut_m.Draw()

c_leading_e = ROOT.TCanvas()
h_leading_e.Draw()
c_leading_e.Draw()

c_2d_leading_e_nu_e = ROOT.TCanvas()
h_2d_leading_e_nu_e.Draw()
c_2d_leading_e_nu_e.Draw()